In [100]:
import pandas as pd
import numpy as np
import copy 
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [101]:
df = pd.read_csv('Data/sensor1.csv')

In [102]:
df

,sensor_id,timestamp,temperature,humidity,ohms,moisture,weather_humidity,weather_pressure,weather_temp_dew,weather_temp_dry,weather_wind_dir,weather_wind_speed,weather_wind_max,weather_wind_min,weather_precip_past10min
0,1,2020-08-28 03:00:00+00:00,15.012970,77.00,270.067017,12.990156,95.579013,1004.827315,11.475756,12.131467,108.966365,1.825696,2.818346,NaN,0.000000
1,1,2020-08-28 04:00:00+00:00,15.198558,77.00,261.323128,13.038244,96.114617,1004.332653,12.136401,12.751486,103.050999,2.153497,3.074299,NaN,0.000000
2,1,2020-08-28 05:00:00+00:00,15.384146,77.00,252.579239,13.086331,95.666685,1003.678711,12.646634,13.377194,102.429338,1.645717,2.843965,NaN,0.000000
3,1,2020-08-28 06:00:00+00:00,15.569734,77.00,243.835350,13.134418,94.448831,1003.407445,13.056790,13.834935,111.662148,2.000634,3.346310,NaN,0.000000
4,1,2020-08-28 07:00:00+00:00,15.755322,77.00,235.091461,13.182506,93.943871,1003.311881,13.650739,14.700486,88.541827,1.934695,3.406582,NaN,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57967,42,2020-10-20 19:00:00+00:00,10.695000,99.99,55.961068,15.403181,83.363973,1002.234081,7.042912,9.740160,191.395958,2.868811,5.950202,NaN,0.068418
57968,42,2020-10-20 20:00:00+00:00,10.660000,99.99,56.017153,15.401460,89.845917,1002.017034,7.713235,9.304126,191.618384,2.302741,4.824224,NaN,0.031582
57969,42,2020-10-20 21:00:00+00:00,10.666667,99.99,55.524990,15.416776,92.324956,1001.807850,8.107616,9.218550,188.959264,2.140578,4.475191,NaN,0.000000
57970,42,2020-10-20 22:00:00+00:00,10.673333,99.99,55.032826,15.432091,93.548737,1001.611697,8.526925,9.522056,191.684353,2.046183,4.293633,NaN,0.000000


In [119]:
def split_train_test_val(df, test_split = 0.2, val_split = 0.2, RANDOM_SEED = 42):
    
    idx_train, idx_test = train_test_split([i for i in range(1,43)], test_size=test_split, random_state = RANDOM_SEED)
    idx_only_train, idx_val = train_test_split(idx_train, test_size=val_split, random_state = RANDOM_SEED)
    
    df['Train'] = False
    df['Validation'] = False
    df['Test'] = False
    
    df.loc[df['sensor_id'].isin(idx_train), 'Train'] = True
    df.loc[df['sensor_id'].isin(idx_test), 'Test'] = True
    df.loc[df['sensor_id'].isin(idx_val), 'Validation'] = True
    
    
    return df
    
def set_index(df):
    df['idx']=0
    for idx, grp in enumerate(df.groupby('sensor_id')):
        df.loc[df.sensor_id == grp[0], "idx"] = idx
        
    
    

In [143]:


class data(Dataset):
    
    def __init__(self, path_to_csv, test, val, list_features,scaling = True, fixed_points = False):
        super(data).__init__()
        df = pd.read_csv(path_to_csv)
        df = split_train_test_val(df)
        
        if test:
            self.df = df[df['Test']==True].reset_index()
            del df
            set_index(self.df)

        else:
            if val:
                self.df = df[(df['Train']==True) & (df['Validation']==True)].reset_index()
            else:
                self.df = df[(df['Train']==True) & (df['Validation']==False)].reset_index()
                
            del df
            set_index(self.df)

        self.fixed_points = fixed_points
        self.list_features = list_features
    
    def __getitem__(self, index):
        
        if self.fixed_points:
            nb_point=200
            start = np.random.randint(0,len(self.df[self.df['idx']==index]) - nb_point +1)
            _input = torch.from_numpy(np.array(self.df[self.df['idx']==index][self.list_features][start:start + nb_point]).transpose())[:,:-1]
            target = torch.from_numpy(np.array(self.df[self.df['idx']==index][self.list_features][start:start + nb_point]).transpose())[:,1:]
            
            return _input, target
            
        else:
            _input = torch.from_numpy(np.array(self.df[self.df['idx']==index][self.list_features]).transpose())[:,:-1]
            target = torch.from_numpy(np.array(self.df[self.df['idx']==index][self.list_features]).transpose())[:,1:]
            
            return _input, target
     
    
    def __len__(self):
        return len(self.df.groupby('sensor_id'))
        
        
        
        
        

In [131]:
dataset = data('Data/sensor1.csv', test = False, list_features = ['humidity'], fixed_points=False)

In [132]:
loader = DataLoader(dataset, batch_size=1)

In [133]:
# Shape : [Batch, Features, Time Steps]

test, train = next(iter(loader))
print("Train | ", train.shape, "|", train)
print("Test | ", test.shape, "|", test)

Train |  torch.Size([1, 1, 1292]) | tensor([[[77.0000, 77.0000, 77.0000,  ..., 69.6000, 69.8000, 70.0000]]],
       dtype=torch.float64)
Test |  torch.Size([1, 1, 1292]) | tensor([[[77.0000, 77.0000, 77.0000,  ..., 69.4000, 69.6000, 69.8000]]],
       dtype=torch.float64)


In [134]:
# # input: array-like of shape (n_samples, n_features)

print(train.shape)
train = torch.squeeze(train, 1)
test = torch.squeeze(test, 1)

# Fit to data to range 0 - 1, then transform it.
scaler = MinMaxScaler()
X_train = scaler.fit_transform(train)
X_test = scaler.fit_transform(test)

print(X_train.shape)




torch.Size([1, 1, 1292])
(1, 1292)


In [135]:
X_train = torch.unsqueeze(torch.from_numpy(X_train).permute(1,0), 2)
X_test = torch.unsqueeze(torch.from_numpy(X_test).permute(1,0), 2)

In [150]:
print(X_train.shape)
print(X_test.shape)

torch.Size([1292, 1, 1])
torch.Size([1292, 1, 1])


In [151]:
class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        
        input_size = 1
        latent_1 = 400
        bottleneck = 16
        
        self.lstm1 = nn.LSTM(input_size, latent_1)
        self.lstm2 = nn.LSTM(latent_1,bottleneck)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        output, _ = self.lstm1(x)
        output = self.relu(output)
        output, _ = self.lstm2(output)
        return output
        

class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()
        
        input_size = 1
        latent_1 = 400
        bottleneck = 16
        
        self.lstm1 = nn.LSTM(bottleneck, latent_1)
        self.lstm2 = nn.LSTM(latent_1, input_size)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        output, _ = self.lstm1(x)
        output = self.relu(output)
        output, _ = self.lstm2(output)
        return output
        
    
class Autoencoder(nn.Module):
    def __init__(self, encoder, decoder):
        super(Autoencoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
    
    def forward(self, x):
        x = self.encoder(x.float())
        x = self.decoder(x)
        return x
            

In [152]:

model = Autoencoder(Encoder(), Decoder())
print(model)

Autoencoder(
  (encoder): Encoder(
    (lstm1): LSTM(1, 400)
    (lstm2): LSTM(400, 16)
    (relu): ReLU()
  )
  (decoder): Decoder(
    (lstm1): LSTM(16, 400)
    (lstm2): LSTM(400, 1)
    (relu): ReLU()
  )
)


In [153]:
def scale_and_reshape(_input, target):
    scaler = MinMaxScaler()
    _input = torch.squeeze(_input, 1)
    target = torch.squeeze(target, 1)
    scaler.fit(_input)
    _input = scaler.transform(_input)
    target = scaler.transform(target)
    _input = torch.unsqueeze(torch.from_numpy(_input).permute(1,0), 2)
    target = torch.unsqueeze(torch.from_numpy(target).permute(1,0), 2)
    
    return _input, target

def reshape(_input, target):
    _input = _input.permute(2,0,1)
    target = target.permute(2,0,1)
    return _input, targets

In [157]:
def train_model(model, train_dl, val_dl, n_epochs):
    
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    criterion = nn.L1Loss(reduction='sum')

    # Track loss
    
    
    train_losses = []
    val_losses = []
    for epoch in range(n_epochs):
        train_loss = []
        val_loss = []
        model = model.train()
        for idx, (_input, target) in enumerate(train_dl):
            _input, target = scale_and_reshape(_input, target)
            optimizer.zero_grad()
            reconstruction = model(_input)
            loss = criterion(reconstruction, _input)
            loss.backward()
            optimizer.step()
            train_loss.append(loss.item())
            print(f"Epoch : {epoch}, Batch : {idx/len(train_dl)}, Training loss : {loss.item()}")

        model = model.eval()
        
        with torch.no_grad():
            for idx, (_input, target) in enumerate(val_dl):
                _input, target = scale_and_reshape(_input, target)
                reconstruction = model(_input)
                loss = criterion(reconstruction, _input)
                val_losses.append(loss.item())
                print(f"Epoch : {epoch}, Batch : {idx/len(val_dl)}, Validation loss : {loss.item()}")

        train_loss = np.mean(train_loss)
        val_loss = np.mean(val_loss)
        train_losses.append(train_loss)
        val_losses.append(val_loss)

        print(f'Epoch : {epoch}, train loss : {train_loss}, val loss :  {val_loss}')
    
    return model, train_losses, val_losses


In [158]:
train_dataset = data('Data/sensor1.csv', test = False, val=False, list_features = ['humidity'], fixed_points=False)
train_dl = DataLoader(train_dataset, batch_size=1)
val_dataset = data('Data/sensor1.csv', test = False, val=True, list_features = ['humidity'], fixed_points=False)
val_dl = DataLoader(val_dataset, batch_size=1)
model, train_losses, val_losses = train_model(model, train_dl, val_dl, n_epochs=1)

Epoch : 0, Batch : 0.0, Training loss : 0.025147709758989695
Epoch : 0, Batch : 0.038461538461538464, Training loss : 0.016847745104017448
Epoch : 0, Batch : 0.07692307692307693, Training loss : 0.013234457634846195
Epoch : 0, Batch : 0.11538461538461539, Training loss : 0.01032870750487595
Epoch : 0, Batch : 0.15384615384615385, Training loss : 0.008829878526158037
Epoch : 0, Batch : 0.19230769230769232, Training loss : 0.007754812511846758
Epoch : 0, Batch : 0.23076923076923078, Training loss : 0.00653874150406584
Epoch : 0, Batch : 0.2692307692307692, Training loss : 0.0053801880149770874
Epoch : 0, Batch : 0.3076923076923077, Training loss : 0.004326057934775773
Epoch : 0, Batch : 0.34615384615384615, Training loss : 0.0033662546823921605
Epoch : 0, Batch : 0.38461538461538464, Training loss : 0.002493217624078405
Epoch : 0, Batch : 0.4230769230769231, Training loss : 0.0016956115893448542
Epoch : 0, Batch : 0.46153846153846156, Training loss : 0.0009459354741486967
Epoch : 0, Batc

ValueError: too many values to unpack (expected 2)